### Youtube comments analysis

In this project, a dataset of user comments for youtube videos related to animals or pets was analyzed. An attempt was made to identify cat or dog owners based on these comments. Topics important to dog or cat owners was studied, and finally, the video creators with the most viewers that are cat or dog owners was identified.

#### 0. Data Exploration and Cleaning

In [3]:
# read data
df = spark.read.load("/FileStore/tables/animals_comments__1_-034b5.csv", format='csv', header = True, inferSchema = True)
df.show(10)
print('Raw data size', df.count())

+--------------------+------+--------------------+
 creator_name|userid| comment|
+--------------------+------+--------------------+
 Doug The Pug| 87.0|I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0|stop saying get e...|
 Meu Zoológico| 670.0|Tenho uma jiboia ...|
 ojatro|1031.0|I wanna see what ...|
 Tingle Triggers|1212.0|Well shit now Im ...|
Hope For Paws - O...|1806.0|when I saw the en...|
Hope For Paws - O...|2036.0|Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0|Call the teddy Larry|
+--------------------+------+--------------------+
only showing top 10 rows

('Raw data size', 5820035)

In [4]:
# find user with preference of dog and cat
# 'I have a dog' contains 'I have dogs' and typo
from pyspark.sql.functions import size,col,count,when
from pyspark.sql.types import *

# Select potential pet owners
cond = (df["comment"].like("%my dog%") | df["comment"].like("%I have a dog%") | df["comment"].like("%my dogs%") | df["comment"].like("%I have dog%")
        | df["comment"].like("%my cat%") | df["comment"].like("%my cats%") | df["comment"].like("%I have a cat%") | df["comment"].like("%I have cat%") 
        | df["comment"].like("%my puppy%") | df["comment"].like("%my puppies%") | df["comment"].like("%my kitty%") | df["comment"].like("%my kitties%") 
        | df["comment"].like("%I have a kitty%") | df["comment"].like("%I have kitties%") | df["comment"].like("%I have a puppy%") | df["comment"].like("%I have puppies%"))

df_clean = df.withColumn('dog_cat',  cond)

# Data cleaning: remove NULL 
for colume in df_clean.columns:
  df_clean=df_clean.filter(df_clean[colume].isNotNull())
#label 1: cat and dog owner; label 0: Non pet owner
df_clean = df_clean.withColumn('label', col("dog_cat").cast(IntegerType()).cast('double')) 

##### 0.1 Tokenize
In this step, comments are tokenized into a list of words

In [6]:
# data preprocessing 
from pyspark.ml.feature import RegexTokenizer

regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="tokenized", pattern="\\W")
df_clean = regexTokenizer.transform(df_clean)

##### 0.2 Remove stop words
In this step, stop words are dropped since they don't have any meanings

In [8]:
from pyspark.ml.feature import StopWordsRemover

# Define a list of stop words or use default list
remover = StopWordsRemover()
stopwords = remover.getStopWords() 

# Display some of the stop words
stopwords[:10]

Out[ 10 ]: 
[u'i',
 u'me',
 u'my',
 u'myself',
 u'we',
 u'our',
 u'ours',
 u'ourselves',
 u'you',
 u'your']

In [9]:
# Specify input/output columns
remover.setInputCol("tokenized")
remover.setOutputCol("vector_no_stopw")

# Transform existing dataframe with the StopWordsRemover
df_clean = remover.transform(df_clean)

In [10]:
# This step was supposed to remove the rows where 'vector_stemmed' are very short or empty array
# Array length less than 4 (4 is hard coded) are removed

df_clean = df_clean.where(size(col('vector_no_stopw')) > 4)

##### 0.3 Stemming

In [12]:
%sh /home/ubuntu/databricks/python/bin/pip install nltk

Collecting nltk
 Downloading https://files.pythonhosted.org/packages/50/09/3b1755d528ad9156ee7243d52aa5cd2b809ef053a0f31b53d92853dd653a/nltk-3.3.0.zip (1.4MB)
Requirement already satisfied: six in /usr/lib/python2.7/dist-packages (from nltk) (1.10.0)
Building wheels for collected packages: nltk
 Running setup.py bdist_wheel for nltk: started
 Running setup.py bdist_wheel for nltk: finished with status 'done'
 Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built nltk
Installing collected packages: nltk
Successfully installed nltk-3.3
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

In [13]:
# Import stemmer library
from nltk.stem.porter import *

# Instantiate stemmer object
stemmer = PorterStemmer()

# Create stemmer python function
def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        if len(t_stem) > 2:
            out_vec.append(t_stem)       
    return out_vec

# Create user defined function for stemming with return type Array<String>
from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))

# Create new column with vectors containing the stemmed tokens 
df_clean = df_clean.withColumn("vector_stemmed", stemmer_udf("vector_no_stopw"))  

#### 1. Build the classifier 
In this step, a classification model was build to identify cat or dog owner from their comments using logistic regression.
TF-IDF was used to vectorize text. TF-IDF is a feature vectorization method widely used in text mining to reflect the importance of a term to a document in the corpus.
<br>
<br> Results: 
<br> Training set areaUnderROC: 0.94326497772
<br> Testing set areaUnderROC 0.93922450574
<br> Training set accuracy: 0.900138546521
<br> Testing set accuracy 0.894351918899

In [15]:
from pyspark.ml.feature import HashingTF, IDF
from pyspark.sql.functions import col,size,count,when,isnan
from pyspark.sql import *
from functools import reduce

df_clean.na.drop()
hashingTF = HashingTF(inputCol="vector_stemmed", outputCol="tf", numFeatures=200)
featurizedData = hashingTF.transform(df_clean)
featurizedData.na.drop()

featurizedData.withColumn('userid', col('userid').cast('float').cast(IntegerType()))

Out[ 15 ]: DataFrame[creator_name: string, userid: int, comment: string, dog_cat: boolean, label: double, tokenized: array<string>, vector_no_stopw: array<string>, vector_stemmed: array<string>, tf: vector]

In [16]:
idf = IDF(inputCol="tf", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.show()

+--------------------+------+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 creator_name|userid| comment|dog_cat|label| tokenized| vector_no_stopw| vector_stemmed| tf| features|
+--------------------+------+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
 bulletproof| 530.0|stop saying get e...| false| 0.0|[stop, saying, ge...|[stop, saying, ge...|[stop, say, get, ...|(200,[14,39,44,52...|(200,[14,39,44,52...|
 Meu Zoológico| 670.0|Tenho uma jiboia ...| false| 0.0|[tenho, uma, jibo...|[tenho, uma, jibo...|[tenho, uma, jibo...|(200,[22,138,157]...|(200,[22,138,157]...|
 ojatro|1031.0|I wanna see what ...| false| 0.0|[i, wanna, see, w...|[wanna, see, happ...|[wanna, see, happ...|(200,[6,45,72,115...|(200,[6,45,72,115...|
Hope For Paws - O...|1806.0|when I saw the en...| false| 0.0|[when, i, saw, th...|[saw, end, said, ...|[saw, end, said, ...|(200,[13,34,37,48...|(200,[13,34,37,48...|
Hope For Paws - O...|2036.0|Holy crap. That i...| false| 0.0|[holy, crap, that...|[holy, crap, quit...|[holi, crap, quit...|(200,[14,17,37,42...|(200,[14,17,37,42...|
Hope For Paws - O...|2911.0|That mother cat l...| false| 0.0|[that, mother, ca...|[mother, cat, loo...|[mother, cat, loo...|(200,[19,56,74,91...|(200,[19,56,74,91...|
Hope For Paws - O...|2911.0|Its people like H...| false| 0.0|[its, people, lik...|[people, like, ho...|[peopl, like, hop...|(200,[0,53,125,13...|(200,[0,53,125,13...|
 Talking Kitty Cat|2911.0|steve: No wet foo...| false| 0.0|[steve, no, wet, ...|[steve, wet, food...|[steve, wet, food...|(200,[14,22,49,53...|(200,[14,22,49,53...|
 Brave Wilderness|3224.0|Dont call this a ...| false| 0.0|[dont, call, this...|[dont, call, chal...|[dont, call, chal...|(200,[3,58,109,14...|(200,[3,58,109,14...|
 Brave Wilderness|3466.0|there is no safe ...| false| 0.0|[there, is, no, s...|[safe, way, hold,...|[safe, way, hold,...|(200,[13,42,53,87...|(200,[13,42,53,87...|
 Brave Wilderness|3466.0|Red before yellow...| false| 0.0|[red, before, yel...|[red, yellow, kil...|[red, yellow, kil...|(200,[28,91,110,1...|(200,[28,91,110,1...|
 Brian Barczyk|3589.0|Im 7 and I love y...| false| 0.0|[im, 7, and, i, l...|[im, 7, love, vid...|[love, videosand,...|(200,[1,40,106],[...|(200,[1,40,106],[...|
 Info Marvel|4504.0|Falto deadpool en...| false| 0.0|[falto, deadpool,...|[falto, deadpool,...|[falto, deadpool,...|(200,[63,106,148]...|(200,[63,106,148]...|
 Gohan The Husky|4533.0|2:35 Why do you l...| false| 0.0|[2, 35, why, do, ...|[2, 35, leave, tv...| [leav, home]|(200,[165,181],[1...|(200,[165,181],[3...|
 Kitsune Kreations|8857.0|I will not put ly...| false| 0.0|[i, will, not, pu...|[put, lyrics, m, ...|[put, lyric, glad...|(200,[17,18,62,65...|(200,[17,18,62,65...|
 Angel Hawk|8857.0|Is Vural a Serval...| false| 0.0|[is, vural, a, se...|[vural, serval, h...|[vural, serval, h...|(200,[55,109,130,...|(200,[55,109,130,...|
 Kitsune Kreations|8857.0|Part 36 Graystrip...| false| 0.0|[part, 36, grayst...|[part, 36, grayst...|[part, graystrip,...|(200,[22,51,66,90...|(200,[22,51,66,90...|
 Abbey Normal|9312.0|did the animal co...| false| 0.0|[did, the, animal...|[animal, come, as...|[anim, come, ask,...|(200,[1,81,112,11...|(200,[1,81,112,11...|
 HoppingHammy|9431.0|its ok girl we wo...| false| 0.0|[its, ok, girl, w...|[ok, girl, rather...|[girl, rather, wa...|(200,[29,37,54,77...|(200,[29,37,54,77...|
 Taylor Nicole Dean|9431.0|omg you can see c...| false| 0.0|[omg, you, can, s...|[omg, see, casey,...|[omg, see, casey,...|(200,[60,77,113,1...|(200,[60,77,113,1...|
+--------------------+------+--------------------+-------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [17]:
# Cat or dog owner are only a small portion of the entire dataset. Building a claasification model using this imbalanced dataset would result in model having a tendency to predict 'non cat or dog owner'. To solve this problem, we randomly selected  some samples from 'non cat or dog owner' with the same size as 'cat or dog owner'.

# The selected data prepared for training are radomly splitted into 80 and 20 percent. 80% for training and the rest 20% for testing
pet = rescaledData.filter("label=1.0")
pet_train, pet_test = pet.randomSplit([0.8, 0.2])
nopet = rescaledData.filter("label=0.0")
sampleRatio = float(pet.count()) / float(nopet.count())
sample_nopet = nopet.sample(False, sampleRatio)
df_sample = pet.unionAll(sample_nopet)
sample_nopet_train, sample_nopet_test = sample_nopet.randomSplit([0.8, 0.2])

df_train = pet_train.unionAll(sample_nopet_train)
df_test = pet_test.unionAll(sample_nopet_test)
print ('training size',df_train.count())
print ('testing size',df_test.count())

('training size', 55577)
('testing size', 13810)

In [18]:
# In this step, use 5-fold crossvalidation to select the best l2 parameter, best model will be saved as 'best_model'
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit

lr = LogisticRegression(maxIter=10,featuresCol='features', labelCol='label')

paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.2, 0.4]) \
    .build()

evaluator=BinaryClassificationEvaluator()
crossval = CrossValidator(estimator = lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=5)

cvModel = crossval.fit(df_train)
best_model = cvModel.bestModel
trainingSummary = best_model.summary

In [19]:
# #Save trained model
path = "/FileStore/tables/PythonLDA/"

best_model.save(path + 'best_model')

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-393305162559119> in <module> () 
 2 path = "/FileStore/tables/PythonLDA/" 
 3 
 ----> 4 best_model . save ( path + 'best_model' ) 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 202 def save ( self , path ) : 
 203 """Save this ML instance to the given path, a shortcut of 'write().save(path)'.""" 
 --> 204 self . write ( ) . save ( path ) 
 205 
 206 

 /databricks/spark/python/pyspark/ml/util.py in save (self, path) 
 163 if not isinstance ( path , basestring ) : 
 164 raise TypeError ( "path should be a basestring, got type %s" % type ( path ) ) 
 --> 165 self . _jwrite . save ( path ) 
 166 
 167 def overwrite ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o2911.save.
: java.io.IOException: Path /FileStore/tables/PythonLDA/best_model already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:503)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:102)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [20]:
# If model has been already trained, load from path
from pyspark.ml.classification import LogisticRegressionModel

path = "FileStore/tables/PythonLDA/"
best_model = LogisticRegressionModel.load(path + 'best_model')


In [21]:
# trainingSummary = best_model.summary
prediction_train = best_model.transform(df_train)
prediction_test = best_model.transform(df_test)
accuracy_train = prediction_train.filter(prediction_train.label == prediction_train.prediction).count()/float(df_train.count())
accuracy_test = prediction_test.filter(prediction_test.label == prediction_test.prediction).count()/float(df_test.count())

print('Training set areaUnderROC: ' + str(evaluator.evaluate(prediction_train)))
print('Testing set areaUnderROC ' + str(evaluator.evaluate(prediction_test)))
print('Training set accuracy: ' + str(accuracy_train))
print('Testing set accuracy ' + str(accuracy_test))

Training set areaUnderROC: 0.94326497772
Testing set areaUnderROC 0.93922450574
Training set accuracy: 0.900138546521
Testing set accuracy 0.894351918899

#### 2. Classify All The Users
We can now apply the cat/dog classifiers to all the other users in the dataset.
The ratio of predicted cat or dog owners are around 14% of the entire population

In [23]:
#Apply trained model to the entire dataset
prediction = best_model.transform(rescaledData)

total_pet_owner = prediction.filter("prediction = 1.0").count()
total_population = df.select("userid").distinct().count()
pet_owner_ratio = float(total_pet_owner)/float(total_population)
print('total_pet_owner :',total_pet_owner)
print('total_population :',total_population)
print('pet_owner_ratio :',pet_owner_ratio)

('total_pet_owner :', 373778)
('total_population :', 2537174)
('pet_owner_ratio :', 0.1473206015827058)

#### 3. Get insigts of Users
In this part, we use LDA to analyze the potential topics that cat or dog owners are also interested in.

The list of potential topics they are interested are:
<br>Fish 
<br>Rabbit 
<br>Chiken 
<br>Snake
<br>Deer
<br>Horse
<br>Hamster
<br>Train

In [25]:
from pyspark.mllib.linalg import Vector, Vectors
from pyspark.ml.clustering import LDA
from pyspark.ml.feature import CountVectorizer, CountVectorizerModel

pet_owner = prediction.filter("prediction = 1.0").select('userid','vector_stemmed')

cv = CountVectorizer(inputCol="vector_stemmed", outputCol="features",
                     minTF=2, # minium number of times a word must appear in a document
                     minDF=4) # minimun number of documents a word must appear in

countVectorModel = cv.fit(pet_owner)

countVectors = (countVectorModel
                .transform(pet_owner)
                .select("userid", "features").cache())

print(len(countVectorModel.vocabulary))  # how many documents, vocab size

numTopics = 10 # number of topics

lda = LDA(k = numTopics,
          maxIter = 50 # number of iterations
          )

ldaModel = lda.fit(countVectors)


# Print topics and top-weighted terms
topics = ldaModel.describeTopics(maxTermsPerTopic=20)
vocabArray = countVectorModel.vocabulary

ListOfIndexToWords = udf(lambda wl: list([vocabArray[w] for w in wl]))
FormatNumbers = udf(lambda nl: ["{:1.4f}".format(x) for x in nl])

topics.select(ListOfIndexToWords(topics.termIndices).alias('words')).show(truncate=False, n=numTopics)

27746
+---------------------------------------------------------------------------------------------------------------------------------------------+
words |
+---------------------------------------------------------------------------------------------------------------------------------------------+
[thank, realli, help, tank, much, fish, turtl, nigga, sweet, carri, hola, cocain, infochammel, tiger, viralhog, jump, ruin, donat, buy, lion]|
[cat, like, fuck, big, great, babi, meow, rabbit, box, white, fish, bite, look, chicken, tree, robin, hand, one, purr, rescu] |
[anim, like, get, dont, know, peopl, want, one, time, year, kitten, think, got, name, also, good, live, take, thing, food] |
[video, kitti, loki, snake, deer, nice, boot, entri, sound, watch, made, sylvest, chi, pee, mishka, chuy, mine, steve, swim, tweet] |
[see, hors, use, eat, human, watch, beauti, walk, that, talk, eye, life, game, hate, funko, omg, way, toy, robin, wish] |
[love, one, look, guy, hope, water, giveaway, win, hamster, call, girl, vid, sleep, amaz, boy, fox, intro, bath, staci, bed] |
[pleas, que, cute, man, por, con, sonic, era, para, mucho, una, todo, est, pero, ador, laugh, como, brush, super, desd] |
[get, pit, dog, littl, cri, play, bull, day, channel, back, servic, leg, ear, vlog, kid, run, pack, famili, subscrib, awesom] |
[need, puppi, dog, pet, pitbul, breed, die, poor, best, save, owner, plz, mom, never, put, new, tail, hit, like, leav] |
[dog, train, peopl, make, happi, treat, kill, owner, rescu, gohan, abus, breed, bad, attack, coyot, fight, bark, aggress, sad, person] |
+---------------------------------------------------------------------------------------------------------------------------------------------+

#### 4. Identify Creators With Cat And Dog Owners In The Audience

'The Dodo' is the creator with largest distinct cat or dog owner audience population, followed by 'brave wilderness' and 'hope for paws'

In [27]:
from pyspark.sql.functions import countDistinct
tmp = prediction.filter("prediction = 1.0")
tmp.groupBy('creator_name').agg(countDistinct('userid')).sort('count(DISTINCT userid)',ascending= False).show()

+--------------------+----------------------+
 creator_name|count(DISTINCT userid)|
+--------------------+----------------------+
 The Dodo| 26027|
 Brave Wilderness| 17711|
Hope For Paws - O...| 14561|
 Robin Seplut| 13890|
 Taylor Nicole Dean| 11294|
 Vet Ranch| 11039|
 Cole & Marmalade| 10045|
 Gohan The Husky| 10017|
 Brian Barczyk| 9594|
 Viktor Larkhill| 7572|
 Paws Channel| 7087|
 stacyvlogs| 6571|
Gone to the Snow ...| 6449|
 Talking Kitty Cat| 5944|
Zak Georges Dog T...| 5442|
 meow meow| 3431|
 ViralBe| 3386|
 The Pet Collective| 3291|
 SlideShow ForFun| 3231|
 Info Marvel| 3188|
+--------------------+----------------------+
only showing top 20 rows

#### 5. Analysis and Future work

According to the work, around 14.5% of the total user who commented on Youtube in this dataset are dog or cat owners. The potential topics they are interested in are include Fish, Rabbit, Chiken, Snake, Deer, Horse, Hamster, Train, etc. Videos related to these topics could be promoted to these cat or dog owners. Also, 'The Dodo', 'brave wilderness' and 'hope for paws' are the top three creators with largest distinct cat or dog owner audience population. Ads targeting cat or dog owners will potentially have the biggest payback cooperating with these creators.  

For future work, this work could be improved in the following aspects:
<br> 1. To select the cat or dog owners more accurately, the breeds of cats and dogs should be considered
<br> 2. A pipeline could be built to replace the stemming, tokenize, tf-idf process
<br> 3. Stop words could be further removed from LDA results